In [ ]:
import os
import openai
import anthropic
import together
import subprocess
import re

# open ai API key
openai_api_key=''
# anthropic API key
claude_api_key = ''
# together AI API key
together_api_key = ''

In [ ]:
# read the prompt from the prompt text file titled "tableallocation.txt"
with open("../prompts/tableallocation.txt", "r") as f:
    prompt = f.read()

prompt = str(prompt)
print(prompt)

In [3]:
claude_client = anthropic.Anthropic(api_key = claude_api_key)
openai_client = openai.OpenAI(api_key = openai_api_key)
togetherai_client = together.Together(api_key=together_api_key)

## GPT-4o

In [4]:
completion = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [5]:
print(response)

To tackle the problem of maximizing total expected revenue in a restaurant with constraints on table capacity and seating arrangement, we can approach it using stochastic simulation. This method will allow us to capture the randomness inherent in customer arrivals and service times, given the constraints of our restaurant setup.

### 1. Explanation of Approach

The problem involves several key components:
- **Poisson Arrivals**: Customers arrive in groups according to a Poisson process, characterized by different arrival rates for each group size.
- **Table Allocation**: We seek to decide the number of tables of each size that optimally balances between seating capacity and potential revenue.
- **Service Time**: Service times follow an exponential distribution.
- **Revenue Model**: Revenue is earned only when a group is successfully seated.

#### Approach Breakdown:
1. **Simulation Framework**:
   - Simulate customer arrivals for each group size using the Poisson process.
   - Assign a

In [6]:
import numpy as np

# Constants
n_hour = 3
capacity = 80
table_cap = [2, 4, 6, 8]
lambda_arrivals = [3, 6, 3, 3, 2, 1.333, 1.2, 1]
service_time_means = [20, 25, 30, 35, 40, 45, 50, 60]
table_revenue = [15, 30, 45, 60, 75, 90, 105, 120]

# Function to simulate the restaurant operation
def simulate_restaurant(num_tables, capacity, n_hour, lambda_arrivals):
    total_revenue = 0
    served_groups = 0
    total_groups = 0

    # Initialize tables
    tables = {cap: [0] * num for cap, num in zip(table_cap, num_tables)}

    # Calculate total simulation time in minutes
    total_time = n_hour * 60
    current_time = 0

    while current_time < total_time:
        # Increment current time by a small unit (say 1 minute)
        current_time += 1

        # For each group size, simulate arrivals
        for group_size in range(1, 9):
            if np.random.poisson(lambda_arrivals[group_size - 1]) > 0:
                # Try to seat the group
                seat_found = False
                for cap in table_cap:
                    if group_size <= cap and sum(tables[cap]) < num_tables[table_cap.index(cap)]:
                        # Seat the group
                        service_time = np.random.exponential(service_time_means[group_size - 1])
                        tables[cap][tables[cap].index(0)] = current_time + service_time
                        total_revenue += table_revenue[group_size - 1]
                        seat_found = True
                        served_groups += 1
                        break

                if not seat_found:
                    # Group leaves if no table is available
                    total_groups += 1
                    continue

        # Free tables whose service time is over
        for cap in table_cap:
            tables[cap] = [0 if t <= current_time else t for t in tables[cap]]

        total_groups += 1  # Assume an arrival attempt for counting purposes

    service_rate = served_groups / total_groups if total_groups > 0 else 0
    return total_revenue, service_rate

# Perform the simulation with the initial guess
num_tables_initial = [10, 5, 4, 2]

print("Running simulation with initial configuration...")
total_revenue, service_rate = simulate_restaurant(num_tables_initial, capacity, n_hour, lambda_arrivals)
print(f"Initial total revenue: {total_revenue}, Service rate: {service_rate:.2f}")

# Optimization process (simplistic, consider using more advanced methods or heuristics)
# Here we simply run a few variations manually for demonstration purposes
best_revenue = total_revenue
best_config = num_tables_initial

# Example: Adjusting allocations incrementally (simple trial & error)
trial_configs = [
    [12, 4, 3, 2],
    [8, 8, 3, 1],
    [9, 5, 5, 1]
]

for config in trial_configs:
    if sum(np.array(config) * np.array(table_cap)) <= capacity:
        revenue, rate = simulate_restaurant(config, capacity, n_hour, lambda_arrivals)
        print(f"Trial config {config}: Revenue = {revenue}, Service Rate = {rate:.2f}")
        if revenue > best_revenue:
            best_revenue = revenue
            best_config = config

print(f"Best configuration: {best_config} with total revenue = {best_revenue}")

Running simulation with initial configuration...
Initial total revenue: 795, Service rate: 0.03
Trial config [12, 4, 3, 2]: Revenue = 795, Service Rate = 0.03
Trial config [8, 8, 3, 1]: Revenue = 795, Service Rate = 0.03
Trial config [9, 5, 5, 1]: Revenue = 720, Service Rate = 0.03
Best configuration: [10, 5, 4, 2] with total revenue = 795


## o3-mini

In [8]:
completion = openai_client.chat.completions.create(
    model="o3-mini",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

print(response)

Below is one complete answer that explains our reasoning, presents a simulation‐based “search” for a good allocation and then shows evidence that our chosen allocation earns more revenue (and serves more groups) than the baseline allocation of [10, 5, 4, 2].

─────────────────────────────
1) Explanation of our Approach
─────────────────────────────
A. Problem “Mechanics” and Customer Assignment

• We have N table types with capacities c₁, c₂, …, cₙ and a restaurant “capacity” constraint:
  x₁*c₁ + x₂*c₂ + … + xₙ*cₙ ≤ capacity.
  Here, xᵢ is the number of tables of size cᵢ. Notice that a table of a given size is only “useful” for groups that are at most that size. When a customer group of size j arrives, we “search” through the available tables that are large enough (i.e. having cᵢ ≥ j) and assign the group to the smallest such available table. If none is free at the arrival time, the group leaves and we lose both its potential revenue and service opportunity.
  
• Customer arrivals occ

In [10]:
import numpy as np
import random
import math

# Model parameters
n_hour = 3                              # simulation time in hours (convert to minutes)
simulation_time = n_hour * 60           # simulation time in minutes
capacity = 80
table_cap = [2, 4, 6, 8]                # table sizes available

# Arrival rates for groups of size 1,...,8  (index 0 = group size 1, etc.)
lambda_groups = [3, 6, 3, 3, 2, 1.3333, 1.2, 1]  
service_time_means = [20, 25, 30, 35, 40, 45, 50, 60]   # in minutes
table_revenue = [15, 30, 45, 60, 75, 90, 105, 120]       # revenue per group

# Baseline candidate allocation given as [x1, x2, x3, x4] (small to large tables)
baseline_allocation = [10, 5, 4, 2]

# Function to generate arrival events for each group size.
def generate_arrivals(sim_time, lam):
    """Return sorted list of arrival times for a Poisson process over sim_time using rate lam."""
    arrivals = []
    t = 0
    while t < sim_time:
        # exponential interarrival time for rate lam
        t += np.random.exponential(1/lam)
        if t < sim_time:
            arrivals.append(t)
    return arrivals

def simulate_one_run(num_tables, seed=None):
    """
    Simulate one run of the restaurant for simulation_time minutes.
    num_tables: a list [x1,x2,...,x4] where x_i is count for table size table_cap[i].
    Returns:
      total_revenue, ratio_served, total_arrivals (number of total groups arrived)
    """
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
    
    # Build list of tables. Each table is a dict with:
    # 'cap': capacity, 'free_time': time when table is free.
    tables = []
    for cap, nt in zip(table_cap, num_tables):
        for i in range(nt):
            tables.append({'cap': cap, 'free_time': 0.0})
            
    # Generate arrival events for each group size from 1 to 8; each event is (arrival_time, group_size)
    events = []
    for j, lam in enumerate(lambda_groups, start=1):
        arrivals = generate_arrivals(simulation_time, lam)
        for t in arrivals:
            events.append((t, j))
    
    # sort events by arrival time
    events.sort(key=lambda x: x[0])
    
    total_revenue = 0.0
    served_count = 0
    total_arrivals = len(events)
    
    for arrival_time, group_size in events:
        # For group size j, service time mean from service_time_means list index= j-1
        service_mean = service_time_means[group_size - 1]
        
        # Find available tables with capacity >= group_size at arrival_time.
        # We choose the one with the smallest table size (to “waste” as little capacity as possible).
        candidate_table = None
        candidate_cap = None
        for table in tables:
            if table['cap'] >= group_size and table['free_time'] <= arrival_time:
                # choose table with smallest capacity among available ones.
                if candidate_table is None or table['cap'] < candidate_cap:
                    candidate_table = table
                    candidate_cap = table['cap']
        if candidate_table is not None:
            # Seat the group
            served_count += 1
            total_revenue += table_revenue[group_size - 1]
            # Draw service time from exponential distribution (in minutes)
            service_time = np.random.exponential(service_mean)
            candidate_table['free_time'] = arrival_time + service_time
        # else: group leaves (lost revenue)
        
    service_rate = served_count / total_arrivals if total_arrivals > 0 else 0
    return total_revenue, service_rate, total_arrivals

def evaluate_configuration(num_tables, replications=5):
    """
    Evaluate a given configuration by averaging over several simulation runs.
    Returns (avg_revenue, avg_service_rate).
    """
    revenues = []
    service_rates = []
    for rep in range(replications):
        rev, sr, _ = simulate_one_run(num_tables, seed=(rep+hash(str(num_tables))%1000))
        revenues.append(rev)
        service_rates.append(sr)
    return np.mean(revenues), np.mean(service_rates)

# Check if a candidate configuration meets the capacity constraint:
def is_feasible(num_tables):
    total_seats = sum(x * cap for x, cap in zip(num_tables, table_cap))
    return total_seats <= capacity

# Random candidate generation: generate a candidate allocation num_tables = [x1, x2, x3, x4]
def random_candidate():
    # determine maximum possible tables per type if used alone
    max_tables = [capacity // cap for cap in table_cap]
    while True:
        candidate = [random.randint(0, m) for m in max_tables]
        if is_feasible(candidate):
            return candidate

# We'll run a Monte Carlo search over up to [BUDGET] candidate configurations.
BUDGET = 1000
best_config = None
best_rev = -1
results = []
for i in range(BUDGET):
    candidate = random_candidate()
    avg_rev, avg_sr = evaluate_configuration(candidate, replications=5)
    results.append((candidate, avg_rev, avg_sr))
    if avg_rev > best_rev:
        best_rev = avg_rev
        best_config = candidate

# Evaluate the baseline allocation
baseline_rev, baseline_sr = evaluate_configuration(baseline_allocation, replications=5)

print("Baseline allocation: {} -> Avg Revenue: {:.2f}, Service Rate: {:.2f}".format(
    baseline_allocation, baseline_rev, baseline_sr))
print("Best found allocation: {} -> Avg Revenue: {:.2f}, Service Rate: {:.2f}".format(
    best_config, best_rev, [r for (c,r,s) in results if c==best_config][0]))

# For transparency, also list a few candidates that performed well:
results_sorted = sorted(results, key=lambda x: x[1], reverse=True)
print("\nTop 5 candidate configurations (allocation, revenue, service_rate):")
for cand, rev, sr in results_sorted[:5]:
    print("Allocation: {}, Revenue: {:.2f}, Service Rate: {:.2f}".format(cand, rev, sr))

# run the best configuration 1000 times to get mean and std deviation of revenue
def run_best_configuration(best_config, replications=1000):
    revenues = []
    for rep in range(replications):
        rev, _, _ = simulate_one_run(best_config, seed=(rep+hash(str(best_config))%1000))
        revenues.append(rev)
    mean_revenue = np.mean(revenues)
    std_deviation = np.std(revenues)
    return mean_revenue, std_deviation
mean_revenue, std_deviation = run_best_configuration(best_config, replications=1000)
print(f"\nRunning best configuration {best_config} for 1000 replications...")
print(f"Mean Revenue: {mean_revenue:.2f}, Standard Deviation: {std_deviation:.2f}")


Baseline allocation: [10, 5, 4, 2] -> Avg Revenue: 5361.00, Service Rate: 0.04
Best found allocation: [6, 17, 0, 0] -> Avg Revenue: 6324.00, Service Rate: 6324.00

Top 5 candidate configurations (allocation, revenue, service_rate):
Allocation: [6, 17, 0, 0], Revenue: 6324.00, Service Rate: 0.05
Allocation: [26, 1, 1, 0], Revenue: 6129.00, Service Rate: 0.06
Allocation: [9, 11, 1, 0], Revenue: 5325.00, Service Rate: 0.04
Allocation: [9, 5, 0, 5], Revenue: 5304.00, Service Rate: 0.04
Allocation: [12, 5, 2, 1], Revenue: 5244.00, Service Rate: 0.04

Running best configuration [6, 17, 0, 0] for 1000 replications...
Mean Revenue: 6031.48, Standard Deviation: 435.39


## o1

In [11]:
completion = openai_client.chat.completions.create(
    model="o1",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [12]:
print(response)

Below is a suggested approach to solving this multi‐table seat‐allocation problem under Poisson arrivals, along with a sample Python simulation that can evaluate candidate allocations and perform a (very) simple random search or local search. The solution is divided into three parts:

1) Explanation / Heuristic Approach  
2) Single-Block Python Code  
3) Example Results and a Sample Recommended Allocation  

────────────────────────────────────────────────────────────────────────
1) EXPLANATION OF APPROACH
────────────────────────────────────────────────────────────────────────

A. General Queueing/Blocking Context  
We have groups of sizes j = 1,…,8 arriving as independent Poisson processes with rates λ₁,…,λ₈. Each group j has exponential service time with mean sʲ (from the service_time_means array), and if no table of capacity ≥ j is free at arrival, that group is lost (blocked). The revenue for serving a group of size j is table_revenue[j−1].  

If we attempted a purely analytical s

In [15]:
import random
import math
import numpy as np

def simulate_allocation(num_tables, 
                        table_cap, 
                        service_time_means,
                        table_revenue,
                        arrival_rates,
                        n_hour=3,
                        n_reps=5,
                        seed=None):
    """
    Simulate the restaurant operation for a given table allocation.
    Returns the average total revenue and average service rate (fraction of arrivals served)
    over n_reps independent replications.
    """
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
    
    # Convert hours to minutes (because our service_time_means are in minutes)
    total_time = n_hour * 60.0
    
    # Pre-create a list of (capacity) for each individual table
    # e.g. if num_tables=[10,5,4,2] and table_cap=[2,4,6,8], we'll have 10 tables of 2, 5 of 4, etc.
    all_tables = []
    for i, x in enumerate(num_tables):
        all_tables.extend([table_cap[i]] * x)
    
    # For repeated replications, store results
    rep_revenues = []
    rep_service_rates = []
    
    # We will do multiple replications to smooth out randomness
    for rep in range(n_reps):
        # Keep track of tables' "next_free_time"
        # next_free_time[t] = time at which table t becomes free
        # Initially all tables are free at time=0
        next_free_time = [0.0] * len(all_tables)
        
        # Totals for this replication
        total_revenue = 0.0
        groups_arrived = 0
        groups_served = 0
        
        # Generate arrival events for each group size j=1..8
        # Instead of thinning, we can generate for each j separately:
        events = []
        for j, lam in enumerate(arrival_rates, start=1):
            # Poisson( lam * total_time ) arrivals in [0, total_time]
            # Number of arrivals:
            n_arrivals_j = np.random.poisson(lam * n_hour)  # lam is "per hour", times n_hour
            # Generate arrival times uniformly in [0, total_time]
            arrival_times_j = np.random.rand(n_arrivals_j) * total_time
            # For each arrival event, store (arrival_time, group_size=j)
            for arrival_t in arrival_times_j:
                events.append((arrival_t, j))
        
        # Sort events by arrival time
        events.sort(key=lambda e: e[0])
        
        # Process arrivals in chronological order
        for (arrival_time, group_size) in events:
            groups_arrived += 1
            # Find the first free table that can accommodate this group
            # in ascending order of table capacity
            candidate_tables = []
            for t_idx, cap in enumerate(all_tables):
                # Must be big enough capacity
                if cap >= group_size:
                    # Must be free by "arrival_time"
                    if next_free_time[t_idx] <= arrival_time:
                        candidate_tables.append((cap, t_idx))
            
            if len(candidate_tables) == 0:
                # No suitable table is free => group blocked
                continue
            else:
                # We seat them at the smallest capacity table among those free
                candidate_tables.sort(key=lambda x: x[0])  # sort by capacity
                chosen_table_cap, chosen_table_idx = candidate_tables[0]
                
                # Seat the group
                groups_served += 1
                # Revenue
                total_revenue += table_revenue[group_size - 1]
                # Generate a departure time
                mean_service = service_time_means[group_size - 1]
                # Exponential with mean = mean_service
                service_duration = np.random.exponential(mean_service)
                departure_time = arrival_time + service_duration
                # Mark the table as busy until departure_time
                next_free_time[chosen_table_idx] = departure_time
        
        rep_revenues.append(total_revenue)
        rep_service_rates.append(groups_served / groups_arrived if groups_arrived>0 else 0)
    
    # Return average across n_reps
    return (np.mean(rep_revenues), np.mean(rep_service_rates))


def random_search(table_cap,
                  capacity,
                  arrival_rates,
                  service_time_means,
                  table_revenue,
                  n_hour=3,
                  n_seeds=1,
                  max_iter=100):
    """
    Performs a naive random search over feasible allocations.
    max_iter: how many random allocations we test
    n_seeds: how many replication seeds per allocation to average
    
    Returns the best allocation found and its performance.
    """
    best_allocation = None
    best_revenue = -1
    best_service_rate = 0.0
    
    # We can fix or vary seeds to reduce or measure variance
    seeds = [100 + i for i in range(n_seeds)]
    
    # A helper to check feasibility
    def feasible(x):
        total_seats = sum(x[i] * table_cap[i] for i in range(len(table_cap)))
        return (total_seats <= capacity)
    
    # We'll include the given example [10,5,4,2] as a baseline
    # and then search random allocations around it.
    candidate0 = [10, 5, 4, 2]
    if feasible(candidate0):
        avg_rev, avg_sr = 0.0, 0.0
        for sd in seeds:
            r, sr = simulate_allocation(candidate0,
                                        table_cap,
                                        service_time_means,
                                        table_revenue,
                                        arrival_rates,
                                        n_hour=n_hour,
                                        n_reps=1,
                                        seed=sd)
            avg_rev += r
            avg_sr += sr
        avg_rev /= len(seeds)
        avg_sr /= len(seeds)
        best_allocation = candidate0
        best_revenue = avg_rev
        best_service_rate = avg_sr
    
    # Now do random search
    for _ in range(max_iter):
        # Sample random x_i up to some limit
        # We'll pick x_i from 0..(capacity/table_cap[i]) or so
        x_candidate = []
        for cap_i in table_cap:
            max_tables_i = capacity // cap_i
            x_candidate.append(random.randint(0, max_tables_i))
        
        if not feasible(x_candidate):
            continue
        
        # Evaluate
        avg_rev, avg_sr = 0.0, 0.0
        for sd in seeds:
            r, sr = simulate_allocation(x_candidate,
                                        table_cap,
                                        service_time_means,
                                        table_revenue,
                                        arrival_rates,
                                        n_hour=n_hour,
                                        n_reps=1,
                                        seed=sd)
            avg_rev += r
            avg_sr += sr
        avg_rev /= len(seeds)
        avg_sr /= len(seeds)

        # Compare to best so far
        if avg_rev > best_revenue:
            best_revenue = avg_rev
            best_allocation = x_candidate
            best_service_rate = avg_sr
    
    return best_allocation, best_revenue, best_service_rate


if __name__ == "__main__":
    # Default parameters:
    n_hour = 3
    capacity = 80
    table_cap = [2, 4, 6, 8]
    arrival_rates = [3, 6, 3, 3, 2, 1.3333, 1.2, 1.0]  # per hour
    service_time_means = [20, 25, 30, 35, 40, 45, 50, 60]  # in minutes
    table_revenue = [15, 30, 45, 60, 75, 90, 105, 120]
    
    # Run the random search
    best_alloc, best_rev, best_sr = random_search(table_cap=table_cap,
                                                  capacity=capacity,
                                                  arrival_rates=arrival_rates,
                                                  service_time_means=service_time_means,
                                                  table_revenue=table_revenue,
                                                  n_hour=n_hour,
                                                  n_seeds=3,
                                                  max_iter=100)
    print("Best Allocation Found:", best_alloc)
    print("Expected Revenue:", best_rev)
    print("Service Rate:", best_sr)

    # Run the best configuration 1000 times to get mean and std deviation of revenue
    mean_revenue, std_deviation = run_best_configuration(best_alloc, replications=1000)
    print(f"\nRunning best configuration {best_alloc} for 1000 replications...")
    print(f"Mean Revenue: {mean_revenue:.2f}, Standard Deviation: {std_deviation:.2f}")

Best Allocation Found: [10, 5, 4, 2]
Expected Revenue: 2470.0
Service Rate: 0.9496199289432372

Running best configuration [10, 5, 4, 2] for 1000 replications...
Mean Revenue: 5632.44, Standard Deviation: 449.67


## Claude Sonnet

In [16]:
response = claude_client.messages.create(
    model="claude-3-5-sonnet-20241022",  # Or another Claude 3 model
    max_tokens=4096,
    system="You are an expert in probability theory and stochastic modeling.",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(response.content[0].text)

I'll analyze this queueing system and propose a solution approach. Let me break this down systematically.

1) ANALYTICAL INSIGHTS:
This is a complex multi-class queueing system with:
- Multiple arrival streams (Poisson)
- Different table sizes
- Flexible seating rules (can seat smaller groups at larger tables)
- Lost sales (turned away customers)

A closed-form solution is challenging due to:
- State space explosion (tracking occupancy of different table types)
- Complex seating rules
- Non-standard queueing discipline

Therefore, I'll propose a simulation-based approach with some analytical guidance.

2) SOLUTION STRATEGY:
I'll develop a simulation that:
a) Uses a discrete-event framework
b) Implements priority seating (smallest suitable table first)
c) Incorporates optimization via a guided search

Here's the implementation:

```python
import numpy as np
from heapq import heappush, heappop
import itertools

class RestaurantSimulator:
    def __init__(self, capacity, table_cap, lambda

In [23]:
import numpy as np
from heapq import heappush, heappop
import itertools

class RestaurantSimulator:
    def __init__(self, capacity, table_cap, lambda_vals, service_time_means,
                 table_revenue, n_hour):
        self.capacity = capacity
        self.table_cap = table_cap
        self.lambda_vals = lambda_vals
        self.service_time_means = service_time_means
        self.table_revenue = table_revenue
        self.n_hour = n_hour
        
    def simulate_one_configuration(self, num_tables, seed=None):
        if seed is not None:
            np.random.seed(seed)
            
        # Initialize events queue (time, event_type, group_size, table_idx)
        events = []
        
        # Initialize available tables
        available_tables = []
        for i, count in enumerate(num_tables):
            for j in range(count):
                available_tables.append((self.table_cap[i], i, j))
        
        # Generate initial arrivals
        for group_size, rate in enumerate(self.lambda_vals, 1):
            if rate > 0:
                next_arrival = np.random.exponential(1/rate)
                heappush(events, (next_arrival, 'arrival', group_size, None))
        
        current_time = 0
        total_revenue = 0
        groups_served = 0
        groups_arrived = 0
        
        while current_time < self.n_hour * 60:  # Convert hours to minutes
            time, event_type, group_size, table_info = heappop(events)
            current_time = time
            
            if event_type == 'arrival':
                groups_arrived += 1
                # Generate next arrival for this group size
                next_arrival = current_time + np.random.exponential(1/self.lambda_vals[group_size-1])
                heappush(events, (next_arrival, 'arrival', group_size, None))
                
                # Try to seat the group
                suitable_tables = [(cap, idx, j) for cap, idx, j in available_tables if cap >= group_size]
                if suitable_tables:
                    # Use smallest suitable table
                    table = min(suitable_tables)
                    available_tables.remove(table)
                    
                    # Schedule departure
                    service_time = np.random.exponential(self.service_time_means[group_size-1])
                    departure_time = current_time + service_time
                    heappush(events, (departure_time, 'departure', group_size, table))
                    
                    # Add revenue
                    total_revenue += self.table_revenue[group_size-1]
                    groups_served += 1
                    
            elif event_type == 'departure':
                # Make table available again
                available_tables.append(table_info)
                
        service_rate = groups_served / groups_arrived if groups_arrived > 0 else 0
        return total_revenue, service_rate

    def optimize(self, n_trials=100):
        best_revenue = 0
        best_config = None
        
        # Generate candidate configurations
        def generate_candidates():
            max_tables = [self.capacity // cap for cap in self.table_cap]
            for config in itertools.product(*[range(m + 1) for m in max_tables]):
                if sum(n * c for n, c in zip(config, self.table_cap)) <= self.capacity:
                    yield config
        
        # Try different configurations
        for config in generate_candidates():
            if n_trials <= 0:
                break
                
            revenue, rate = self.simulate_one_configuration(config)
            if revenue > best_revenue:
                best_revenue = revenue
                best_config = config
            n_trials -= 1
            
        return best_config, best_revenue
    
simulator = RestaurantSimulator(
    capacity=80,
    table_cap=[2, 4, 6, 8],
    lambda_vals=[3, 6, 3, 3, 2, 1.333, 1.2, 1],
    service_time_means=[20, 25, 30, 35, 40, 45, 50, 60],
    table_revenue=[15, 30, 45, 60, 75, 90, 105, 120],
    n_hour=3
)

# Run optimization
best_config, best_revenue = simulator.optimize(n_trials=1000)
print(f"Best configuration: {best_config}")
print(f"Expected revenue: ${best_revenue:.2f}")

# Compare with baseline
baseline = [10, 5, 4, 2]
baseline_revenue, baseline_rate = simulator.simulate_one_configuration(baseline)
print(f"Baseline revenue: ${baseline_revenue:.2f}")

# run the best configuration 1000 times to get mean and std deviation of revenue
# there is no function like run_simulation
def run_best_configuration(best_config, replications=1000):
    revenues = []
    for rep in range(replications):
        rev, _ = simulator.simulate_one_configuration(best_config)
        revenues.append(rev)
    mean_revenue = np.mean(revenues)
    std_deviation = np.std(revenues)
    return mean_revenue, std_deviation
mean_revenue, std_deviation = run_best_configuration(best_config, replications=1000)
print(f"\nRunning best configuration {best_config} for 1000 replications...")
print(f"Mean Revenue: {mean_revenue:.2f}, Standard Deviation: {std_deviation:.2f}")



Best configuration: (0, 15, 2, 1)
Expected revenue: $6240.00
Baseline revenue: $5385.00

Running best configuration (0, 15, 2, 1) for 1000 replications...
Mean Revenue: 5150.46, Standard Deviation: 412.93


## DeepSeek

In [25]:
import heapq
import numpy as np

def simulate(num_tables, table_cap, capacity, lambda_rates, service_time_means, table_revenue, T, seed=None):
    if seed is not None:
        np.random.seed(seed)
    # Check capacity constraint
    total_seats = sum(x * c for x, c in zip(num_tables, table_cap))
    if total_seats > capacity:
        return -np.inf, 0.0
    
    # Generate arrival events for each group size (1-8)
    arrival_events = []
    for j in range(1, 9):
        lambda_j = lambda_rates[j-1]
        if lambda_j <= 0:
            continue
        current_time = 0.0
        while True:
            inter_arrival = np.random.exponential(1 / lambda_j)
            current_time += inter_arrival
            if current_time >= T:
                break
            arrival_events.append((current_time, j))
    
    # Sort arrivals by time
    arrival_events.sort()
    
    # Initialize event queue with arrivals
    event_queue = []
    for time, j in arrival_events:
        heapq.heappush(event_queue, (time, 'arrival', j))
    
    available_tables = list(num_tables)
    total_revenue = 0
    served = 0
    total_groups = len(arrival_events)
    
    # Process events
    while event_queue:
        time, event_type, data = heapq.heappop(event_queue)
        
        if event_type == 'arrival':
            j = data
            # Find smallest table that can fit j
            required = None
            for i, cap in enumerate(table_cap):
                if cap >= j and available_tables[i] > 0:
                    required = i
                    break
            if required is not None:
                available_tables[required] -= 1
                served += 1
                total_revenue += table_revenue[j-1]
                # Generate service time
                mean_service = service_time_means[j-1] / 60  # hours
                service_time = np.random.exponential(mean_service)
                heapq.heappush(event_queue, (time + service_time, 'departure', required))
        elif event_type == 'departure':
            table_i = data
            available_tables[table_i] += 1
    
    service_rate = served / total_groups if total_groups > 0 else 0.0
    return total_revenue, service_rate

def generate_neighbors(config, table_cap):
    neighbors = []
    # Swap 1 of table 4 (i=1) to 2 tables of 2 (i=0)
    if config[1] >= 1:
        new_config = config.copy()
        new_config[1] -= 1
        new_config[0] += 2
        neighbors.append(new_config)
    # Swap 1 of table 6 (i=2) to 3 tables of 2 (i=0)
    if config[2] >= 1:
        new_config = config.copy()
        new_config[2] -= 1
        new_config[0] += 3
        neighbors.append(new_config)
    # Swap 1 of table 8 (i=3) to 4 tables of 2 (i=0)
    if config[3] >= 1:
        new_config = config.copy()
        new_config[3] -= 1
        new_config[0] += 4
        neighbors.append(new_config)
    # Swap 1 of table 8 (i=3) to 2 tables of 4 (i=1)
    if config[3] >= 1:
        new_config = config.copy()
        new_config[3] -= 1
        new_config[1] += 2
        neighbors.append(new_config)
    return neighbors

def optimize():
    # Problem parameters
    n_hour = 3
    capacity = 80
    table_cap = [2, 4, 6, 8]
    lambda_rates = [3, 6, 3, 3, 2, 1.3333333333333333, 1.2, 1]
    service_time_means = [20, 25, 30, 35, 40, 45, 50, 60]
    table_revenue = [15, 30, 45, 60, 75, 90, 105, 120]
    initial_config = [10, 5, 4, 2]
    budget = 1000
    n_replicates = 5  # Number of simulations per config to average
    
    current_config = initial_config.copy()
    best_config = current_config.copy()
    best_avg_revenue = 0
    
    # Evaluate initial config
    total = 0
    for _ in range(n_replicates):
        rev, _ = simulate(current_config, table_cap, capacity, lambda_rates, service_time_means, table_revenue, n_hour)
        total += rev
    best_avg_revenue = total / n_replicates
    print(f"Initial config: {initial_config}, Avg Revenue: {best_avg_revenue}")
    budget_used = n_replicates
    
    while budget_used < budget:
        neighbors = generate_neighbors(current_config)
        improved = False
        
        for neighbor in neighbors:
            if budget_used + n_replicates > budget:
                break
            # Check capacity
            total_seats = sum(x * c for x, c in zip(neighbor, table_cap))
            if total_seats != capacity:
                continue
            
            # Evaluate neighbor
            total = 0
            for _ in range(n_replicates):
                rev, _ = simulate(neighbor, table_cap, capacity, lambda_rates, service_time_means, table_revenue, n_hour)
                total += rev
            avg_rev = total / n_replicates
            budget_used += n_replicates
            
            if avg_rev > best_avg_revenue:
                best_avg_revenue = avg_rev
                best_config = neighbor.copy()
                current_config = neighbor.copy()
                improved = True
                print(f"New best: {current_config}, Avg Revenue: {best_avg_revenue}")
                break  # Move to next iteration
        
        if not improved:
            break  # Local optimum
    
    # Final evaluation with more replicates for accuracy
    final_replicates = 50
    total_rev = 0
    total_sr = 0
    for _ in range(final_replicates):
        rev, sr = simulate(best_config, table_cap, capacity, lambda_rates, service_time_means, table_revenue, n_hour)
        total_rev += rev
        total_sr += sr
    avg_rev = total_rev / final_replicates
    avg_sr = total_sr / final_replicates
    
    print("\nFinal Recommendation:")
    print(f"Tables: {best_config}")
    print(f"Total Revenue: {avg_rev:.2f}")
    print(f"Service Rate: {avg_sr:.2%}")

optimize()

Initial config: [10, 5, 4, 2], Avg Revenue: 2982.0


TypeError: generate_neighbors() missing 1 required positional argument: 'table_cap'